In [26]:
import numpy as np

# Load dataset files
synthetic_eeg_eog_path = "/home/tulgaa/Desktop/denoisenet/Linear_Mixing/EEG+EOG/Linear_synthetic_eeg_eog.npy"
clean_eeg_path = "/home/tulgaa/Desktop/denoisenet/Linear_Mixing/EEG+EOG/EEG_all_epochs.npy"

# Load the numpy arrays
synthetic_eeg_eog = np.load(synthetic_eeg_eog_path, allow_pickle=True).item()  # Dictionary of SNR levels
clean_eeg = np.load(clean_eeg_path)  # Ground truth EEG

# Inspect dataset keys and shapes
print("🔹 Synthetic EEG+EOG Dictionary Keys (SNR Levels):", synthetic_eeg_eog.keys())
print("🔹 Clean EEG Shape:", clean_eeg.shape)

# Print shape of each SNR level data
for snr in synthetic_eeg_eog.keys():
    print(f"🔹 SNR Level {snr}: Shape = {synthetic_eeg_eog[snr].shape}")

# Check statistics of clean EEG data
print("🔹 Clean EEG Mean:", np.mean(clean_eeg))
print("🔹 Clean EEG Std Dev:", np.std(clean_eeg))



🔹 Synthetic EEG+EOG Dictionary Keys (SNR Levels): dict_keys([-7, -6, -5, -4, -3, -2, -1, 0, 1, 2])
🔹 Clean EEG Shape: (3400, 512)
🔹 SNR Level -7: Shape = (3400, 512)
🔹 SNR Level -6: Shape = (3400, 512)
🔹 SNR Level -5: Shape = (3400, 512)
🔹 SNR Level -4: Shape = (3400, 512)
🔹 SNR Level -3: Shape = (3400, 512)
🔹 SNR Level -2: Shape = (3400, 512)
🔹 SNR Level -1: Shape = (3400, 512)
🔹 SNR Level 0: Shape = (3400, 512)
🔹 SNR Level 1: Shape = (3400, 512)
🔹 SNR Level 2: Shape = (3400, 512)
🔹 Clean EEG Mean: -0.16774763156951383
🔹 Clean EEG Std Dev: 231.82429071340553


In [27]:
import torch
from sklearn.model_selection import train_test_split

# Lists to store train/test data
X_train_list, X_test_list = [], []
y_train_list, y_test_list = [], []

# Loop through each SNR level
for snr in synthetic_eeg_eog.keys():
    contaminated_signals = synthetic_eeg_eog[snr]  # Shape: (3400, 512)
    clean_signals = clean_eeg  # Shape: (3400, 512) (Assuming same order for each SNR)

    # Split 80% training, 20% testing
    X_train, X_test, y_train, y_test = train_test_split(
        contaminated_signals, clean_signals, test_size=0.2, random_state=42
    )

    # Append to respective lists
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)

# Stack training and testing data
X_train_final = np.vstack(X_train_list)
X_test_final = np.vstack(X_test_list)
y_train_final = np.vstack(y_train_list)
y_test_final = np.vstack(y_test_list)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_final, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_final, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_final, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_final, dtype=torch.float32)

# Print final dataset shapes
print("✅ Training Set Shape:", X_train_tensor.shape, "Target Shape:", y_train_tensor.shape)
print("✅ Testing Set Shape:", X_test_tensor.shape, "Target Shape:", y_test_tensor.shape)


✅ Training Set Shape: torch.Size([27200, 512]) Target Shape: torch.Size([27200, 512])
✅ Testing Set Shape: torch.Size([6800, 512]) Target Shape: torch.Size([6800, 512])


In [28]:
import torch.nn as nn

class EEGDnet(nn.Module):
    def __init__(self, seq_len=512, embed_dim=64, num_heads=4, depth=6, mlp_dim=128):
        super(EEGDnet, self).__init__()
        
        # Reshape Layer: Convert 1D EEG (512,) → 2D (8, 64)
        self.seq_len = seq_len
        self.embed_dim = embed_dim
        self.h, self.w = 8, 64  # Reshaped dimensions

        # Linear Projection to Embed Dim
        self.embedding = nn.Linear(self.w, embed_dim)  # Projects 64 → 64
        self.pos_embedding = nn.Parameter(torch.randn(1, self.h, embed_dim))  # Positional Encoding

        # Transformer Encoder
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=mlp_dim, batch_first=True),
            num_layers=depth
        )

        # Output Layer: Convert back to 1D EEG (512,)
        self.output_layer = nn.Linear(embed_dim, self.w)

    def forward(self, x):
        # Reshape (Batch, 512) → (Batch, 8, 64)
        x = x.view(-1, self.h, self.w)

        # Embedding & Positional Encoding
        x = self.embedding(x) + self.pos_embedding

        # Transformer Encoder
        x = self.transformer(x)

        # Convert back to 1D EEG (Batch, 512)
        x = self.output_layer(x).view(-1, self.seq_len)
        return x

# Initialize Model
model = EEGDnet()
print(model)


EEGDnet(
  (embedding): Linear(in_features=64, out_features=64, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=64, out_features=64, bias=True)
)


In [29]:
import torch.optim as optim

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔥 Using device: {device}")

# Move model to GPU
model = EEGDnet().to(device)

# Define loss function (Mean Squared Error)
criterion = nn.MSELoss()

# Define optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=5e-5)

# Early Stopping Parameters
patience = 10  # Stop if validation loss does not improve for 10 epochs
best_val_loss = float('inf')
epochs_no_improve = 0


🔥 Using device: cuda


In [30]:
import time

def train_model(model, X_train, y_train, X_test, y_test, epochs=100, batch_size=64):
    global best_val_loss, epochs_no_improve
    model.train()
    
    # Move tensors to GPU
    X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)

    # Create Data Loaders
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        start_time = time.time()
        train_loss = 0.0

        # Training loop
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move data to GPU
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Compute average training loss
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation loss calculation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val, y_val in test_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)  # Move data to GPU
                val_output = model(X_val)
                val_loss += criterion(val_output, y_val).item()
        
        avg_val_loss = val_loss / len(test_loader)
        elapsed_time = time.time() - start_time

        # Print training progress
        print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {avg_train_loss:.6f} - Val Loss: {avg_val_loss:.6f} - Time: {elapsed_time:.2f}s")

        # Early Stopping Logic
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), "best_eegdnet.pth")  # Save the best model
            print("✅ Model improved! Saving checkpoint.")
        else:
            epochs_no_improve += 1
            print(f"⚠️ No improvement for {epochs_no_improve} epochs.")

        if epochs_no_improve >= patience:
            print("⏹️ Early stopping triggered!")
            break

        model.train()  # Switch back to training mode

# Start Training with CUDA
train_model(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, epochs=2200, batch_size=1024)


Epoch [1/2200] - Train Loss: 53769.745226 - Val Loss: 53634.494978 - Time: 0.76s
✅ Model improved! Saving checkpoint.
Epoch [2/2200] - Train Loss: 53734.737558 - Val Loss: 53612.439732 - Time: 0.81s
✅ Model improved! Saving checkpoint.
Epoch [3/2200] - Train Loss: 53673.408854 - Val Loss: 53587.070871 - Time: 0.77s
✅ Model improved! Saving checkpoint.
Epoch [4/2200] - Train Loss: 53691.058883 - Val Loss: 53561.040737 - Time: 0.80s
✅ Model improved! Saving checkpoint.
Epoch [5/2200] - Train Loss: 53689.352865 - Val Loss: 53536.657924 - Time: 0.82s
✅ Model improved! Saving checkpoint.
Epoch [6/2200] - Train Loss: 53626.133102 - Val Loss: 53512.135045 - Time: 0.72s
✅ Model improved! Saving checkpoint.
Epoch [7/2200] - Train Loss: 53661.894531 - Val Loss: 53488.588170 - Time: 0.83s
✅ Model improved! Saving checkpoint.
Epoch [8/2200] - Train Loss: 53596.162471 - Val Loss: 53466.815848 - Time: 0.74s
✅ Model improved! Saving checkpoint.
Epoch [9/2200] - Train Loss: 53583.585938 - Val Loss: 53

In [31]:
import torch
import numpy as np
from scipy.signal import welch
import torch.nn.functional as F

def compute_rrmse_t(gt, pred):
    """
    Compute Temporal RRMSE (Lower is better).
    Formula: RRMSE_T = ||pred - gt||_2 / ||gt||_2
    """
    return torch.norm(pred - gt, p=2) / torch.norm(gt, p=2)

def compute_rrmse_s(gt, pred, fs=512):
    """
    Compute Spectral RRMSE (Lower is better).
    Formula: RRMSE_S = ||PSD(pred) - PSD(gt)||_2 / ||PSD(gt)||_2
    """
    def compute_psd(signal):
        f, psd = welch(signal, fs=fs, nperseg=256)  # Power Spectral Density
        return torch.tensor(psd, dtype=torch.float32)

    psd_gt = compute_psd(gt.cpu().numpy())
    psd_pred = compute_psd(pred.cpu().numpy())

    return torch.norm(psd_pred - psd_gt, p=2) / torch.norm(psd_gt, p=2)

def compute_cc(gt, pred):
    """
    Compute Correlation Coefficient (Higher is better).
    Formula: CC = Cov(gt, pred) / (std(gt) * std(pred))
    """
    gt_mean, pred_mean = torch.mean(gt), torch.mean(pred)
    numerator = torch.sum((gt - gt_mean) * (pred - pred_mean))
    denominator = torch.sqrt(torch.sum((gt - gt_mean) ** 2) * torch.sum((pred - pred_mean) ** 2))

    return numerator / (denominator + 1e-8)  # Avoid division by zero


In [32]:
def evaluate_per_snr(model, synthetic_eeg_eog, clean_eeg):
    """
    Evaluate model performance per SNR level using RRMSE_T, RRMSE_S, and CC.
    """
    model.eval()  # Set model to evaluation mode
    
    snr_metrics = {}

    for snr in sorted(synthetic_eeg_eog.keys()):  # Iterate through SNR levels
        X_test_snr = torch.tensor(synthetic_eeg_eog[snr], dtype=torch.float32).to(device)
        y_test_snr = torch.tensor(clean_eeg, dtype=torch.float32).to(device)

        with torch.no_grad():
            y_pred_snr = model(X_test_snr)  # Model prediction (denoised EEG)
        
        # Compute metrics
        rrmse_t = compute_rrmse_t(y_test_snr, y_pred_snr).cpu().item()
        rrmse_s = compute_rrmse_s(y_test_snr, y_pred_snr).cpu().item()
        cc = compute_cc(y_test_snr, y_pred_snr).cpu().item()

        # Store results
        snr_metrics[snr] = {
            "RRMSE_T": rrmse_t,
            "RRMSE_S": rrmse_s,
            "CC": cc
        }

        print(f"SNR {snr}: RRMSE_T = {rrmse_t:.4f}, RRMSE_S = {rrmse_s:.4f}, CC = {cc:.4f}")

    return snr_metrics


In [33]:
def compute_average_metrics(snr_metrics):
    """
    Compute the final average performance of RRMSE_T, RRMSE_S, and CC across all SNRs.
    """
    avg_rrmse_t = sum([snr_metrics[snr]["RRMSE_T"] for snr in snr_metrics]) / len(snr_metrics)
    avg_rrmse_s = sum([snr_metrics[snr]["RRMSE_S"] for snr in snr_metrics]) / len(snr_metrics)
    avg_cc = sum([snr_metrics[snr]["CC"] for snr in snr_metrics]) / len(snr_metrics)
    
    print("\n🔹 **Final Average Results Across All SNRs** 🔹")
    print(f"✅ Average RRMSE_T: {avg_rrmse_t:.4f} (Lower is better)")
    print(f"✅ Average RRMSE_S: {avg_rrmse_s:.4f} (Lower is better)")
    print(f"✅ Average CC: {avg_cc:.4f} (Higher is better)")

    return avg_rrmse_t, avg_rrmse_s, avg_cc

In [34]:
# Evaluate the model on each SNR level
snr_results = evaluate_per_snr(model, synthetic_eeg_eog, clean_eeg)

# Compute and print final averaged metrics
compute_average_metrics(snr_results)


SNR -7: RRMSE_T = 0.5389, RRMSE_S = 0.5517, CC = 0.8508
SNR -6: RRMSE_T = 0.5135, RRMSE_S = 0.5369, CC = 0.8686
SNR -5: RRMSE_T = 0.4957, RRMSE_S = 0.5271, CC = 0.8804
SNR -4: RRMSE_T = 0.4831, RRMSE_S = 0.5208, CC = 0.8885
SNR -3: RRMSE_T = 0.4742, RRMSE_S = 0.5171, CC = 0.8941
SNR -2: RRMSE_T = 0.4680, RRMSE_S = 0.5150, CC = 0.8979
SNR -1: RRMSE_T = 0.4638, RRMSE_S = 0.5140, CC = 0.9005
SNR 0: RRMSE_T = 0.4608, RRMSE_S = 0.5134, CC = 0.9023
SNR 1: RRMSE_T = 0.4587, RRMSE_S = 0.5134, CC = 0.9036
SNR 2: RRMSE_T = 0.4571, RRMSE_S = 0.5136, CC = 0.9046

🔹 **Final Average Results Across All SNRs** 🔹
✅ Average RRMSE_T: 0.4814 (Lower is better)
✅ Average RRMSE_S: 0.5223 (Lower is better)
✅ Average CC: 0.8891 (Higher is better)


(0.4813835173845291, 0.5222980797290802, 0.8891362071037292)